In [2]:
pwd

'C:\\Users\\dipak\\Desktop\\wine\\notebook'

In [4]:
from src.get_data import read_params
import argparse
import mlflow
from mlflow.tracking.client import MlflowClient
from pprint import pprint
import joblib
import os

os.chdir('C:\\Users\\dipak\\Desktop\\wine')


def log_production_model(config_path):
    config = read_params(config_path)

    mlflow_config = config["mlflow_config"]
    model_name = mlflow_config["registered_model_name"]
    remote_server_uri = mlflow_config["remote_server_uri"]
    mlflow.set_tracking_uri(remote_server_uri)


    runs = mlflow.search_runs("1")
    print(runs)
    #print(runs.info.run_id)
    lowest = runs["metrics.mae"].sort_values(ascending=True)[0]
    lowest_run_id = runs[runs["metrics.mae"] == lowest]["run_id"][0]

    client = MlflowClient()
    for mv in client.search_model_versions(f"name='{model_name}'"):
        mv = dict(mv)

        if mv["run_id"] == lowest_run_id:
            current_version = mv["version"]
            logged_model = mv["source"]
            pprint(mv, indent=4)
            client.transition_model_version_stage(name=model_name, version=current_version, stage="Production")
        else:
            current_version = mv["version"]
            client.transition_model_version_stage(name=model_name, version=current_version, stage="Staging")

            
log_production_model("params.yaml")

                             run_id experiment_id    status  \
0  4bd1f88d8b224c80afdec04295780d6b             1  FINISHED   
1  d68a1ddd24154e7fb3308f23a87075c6             1  FINISHED   
2  9c6b61946f90426dbd3f8b12e89d3be6             1  FINISHED   

                                        artifact_uri  \
0  ./artifacts/1/4bd1f88d8b224c80afdec04295780d6b...   
1  ./artifacts/1/d68a1ddd24154e7fb3308f23a87075c6...   
2  ./artifacts/1/9c6b61946f90426dbd3f8b12e89d3be6...   

                        start_time                         end_time  \
0 2021-12-27 15:26:48.830000+00:00 2021-12-27 15:26:53.322000+00:00   
1 2021-12-27 15:26:20.761000+00:00 2021-12-27 15:26:25.025000+00:00   
2 2021-12-27 15:24:40.059000+00:00 2021-12-27 15:24:44.358000+00:00   

   metrics.r2  metrics.mae  metrics.rmse params.alpha params.l1_ratio  \
0    0.109640     0.618863      0.762795          0.5             0.5   
1    0.008382     0.659818      0.805002         0.88            0.89   
2    0.007544     